In [1]:
import numpy as np
import pinocchio as pin



In [2]:
%%capture
!pip3 install robot_descriptions
from robot_descriptions import z1_mj_description, z1_description, skydio_x2_mj_description, skydio_x2_description

In [3]:
# ROBOT_URDF = skydio_x2_description.URDF_PATH
# ROBOT_XML = skydio_x2_mj_description.MJCF_PATH
ROBOT_URDF = z1_description.URDF_PATH
ROBOT_XML =  z1_mj_description.MJCF_PATH


In [4]:
model = pin.buildModelFromUrdf(ROBOT_URDF)
data = model.createData()


In [5]:
q = np.random.randn(model.nq)
if model.nq != model.nv:
    q[3:7]/=np.linalg.norm(q[3:7])
dq = np.random.randn(model.nv)
ddq  = np.random.randn(model.nv)
data.q = q.copy()
data.dq = dq.copy()
data.ddq = ddq.copy()


### Custom Regressor

In [6]:
pin.computeAllTerms(model, data, q, dq)
pin.forwardKinematics(model, data, q, dq, ddq)

def Y_body(v, a_g):
    a1, a2, a3, a4, a5, a6 = a_g
    v1, v2, v3, v4, v5, v6 = v

    Y = np.array([
    [a1 - v2*v6 + v3*v5, -v5**2 - v6**2, -a6 + v4*v5, a5 + v4*v6, 0, 0, 0, 0, 0, 0],
    [a2 + v1*v6 - v3*v4, a6 + v4*v5, -v4**2 - v6**2, -a4 + v5*v6, 0, 0, 0, 0, 0, 0],
    [a3 - v1*v5 + v2*v4, -a5 + v4*v6, a4 + v5*v6, -v4**2 - v5**2, 0, 0, 0, 0, 0, 0],
    [0, 0, a3 - v1*v5 + v2*v4, -a2 - v1*v6 + v3*v4, a4, a5 - v4*v6, -v5*v6, a6 + v4*v5, v5**2 - v6**2, v5*v6],
    [0, -a3 + v1*v5 - v2*v4, 0, a1 - v2*v6 + v3*v5, v4*v6, a4 + v5*v6, a5, -v4**2 + v6**2, a6 - v4*v5, -v4*v6],
    [0, a2 + v1*v6 - v3*v4, -a1 + v2*v6 - v3*v5, 0, -v4*v5, v4**2 - v5**2, v4*v5, a4 - v5*v6, a5 + v4*v6, a6]
])

    return Y


def bodyRegressor(ind):
    """Body regressor
    Derivation is accordingly to the paper: https://arxiv.org/pdf/1610.08703.pdf
    link: https://colab.research.google.com/drive/1xFte2FT0nQ0ePs02BoOx4CmLLw5U-OUZ#scrollTo=FfkIBpi1wLXz
    """
    vel = pin.getVelocity(model,
                    data,
                    ind,
                    pin.LOCAL) 
    
    accel = pin.getAcceleration(model,
                    data,
                    ind,
                    pin.LOCAL)  
    
    rotation = data.oMi[ind].rotation

    v = np.array(vel)
    a = np.array(accel)
    a_g = a 
    a_g[:3] +=  rotation.T @ np.array([0,0,9.81])


    return Y_body(v, a_g)

njoints = model.njoints - 1
force_Y = np.zeros((6*njoints, njoints*10))
col_jac = np.zeros((6*njoints, model.nv))


for i in range(njoints):
    
    jacobian = pin.getJointJacobian(model, 
                                    data, 
                                    i+1, 
                                    pin.LOCAL)
    col_jac[6*i:6*(i+1),:] = jacobian  
    force_Y[6*i:6*(i+1),10*i:10*(i+1)] = bodyRegressor(i+1)#regressor_body(i+1)
    
Ycustom = col_jac.T @force_Y

In [7]:
Ypin = pin.computeJointTorqueRegressor(model,  data, q, dq, ddq)
Ypin

array([[ 6.20672584, -0.51388771, -1.35270205,  0.88693304,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-2.12654381,  0.86615216, -0.84845807, -1.36658705,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-3.99701793, -0.04193924,  0.92214522, -0.59048126,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -3.99701793,  2.12654381,  1.14436614,
         0.04193924,  0.22222092,  0.86615216, -0.25797681, -0.22222092],
       [ 0.        ,  3.99701793,  0.        ,  6.20672584,  0.4224969 ,
         0.92214522,  0.46443614, -0.07659354,  1.35270205, -0.4224969 ],
       [ 0.        , -2.12654381, -6.20672584,  0.        ,  0.24327494,
         0.33457035, -0.24327494,  1.36658705,  0.88693304,  1.1094271 ]])

In [8]:
np.allclose(Ycustom,Ypin)

True

### Regressor in Mujoco

In [9]:
import mujoco

In [11]:
print(ROBOT_XML)

/home/simeon/.cache/robot_descriptions/mujoco_menagerie/skydio_x2/x2.xml


In [12]:
# Load model, make data
mj_model = mujoco.MjModel.from_xml_path(ROBOT_XML)
mj_data = mujoco.MjData(mj_model)
mj_model.opt.enableflags |= mujoco.mjtEnableBit.mjENBL_ENERGY

# model.energy.setflags(enable=True)

# Set the state to the "squat" keyframe, call mj_forward.
# key = model.key('hover').id
# mujoco.mj_resetDataKeyframe(model, data, key)
q_muj = q.copy()
q_muj[[3,4,5,6]] = q[[6,3,4,5]]

mj_data.qpos = q_muj 
mujoco.mj_forward(mj_model, mj_data)



In [13]:
q_muj

array([-0.05734004,  0.54043218,  1.69709864,  0.48406141, -0.06219627,
       -0.8678923 ,  0.09262357])

In [14]:
mj_model.nbody -1

1

In [15]:
mj_model.body_ipos[1]

array([0.        , 0.        , 0.05396226])

In [16]:
mj_data.body('x2')

<_MjDataBodyViews
  cacc: array([ 2.37095592e-16, -1.96884091e-16, -3.25049142e-16, -8.36729246e-17,
        2.99193270e-17,  0.00000000e+00])
  cfrc_ext: array([0., 0., 0., 0., 0., 0.])
  cfrc_int: array([ 8.22889964e-18, -5.99585846e-18, -1.24579242e-17, -1.10866625e-16,
        3.96431083e-17,  0.00000000e+00])
  cinert: array([0.05209638, 0.02628929, 0.04420573, 0.00208754, 0.01141956,
       0.00404512, 0.        , 0.        , 0.        , 1.325     ])
  crb: array([0.05209638, 0.02628929, 0.04420573, 0.00208754, 0.01141956,
       0.00404512, 0.        , 0.        , 0.        , 1.325     ])
  cvel: array([0., 0., 0., 0., 0., 0.])
  id: 1
  name: 'x2'
  subtree_angmom: array([0., 0., 0.])
  subtree_com: array([-0.1033023 ,  0.53500568,  1.66935066])
  subtree_linvel: array([0., 0., 0.])
  xfrc_applied: array([0., 0., 0., 0., 0., 0.])
  ximat: array([-0.80299075,  0.59571083, -0.01828832, -0.11736455, -0.18813774,
       -0.975105  , -0.58432133, -0.78085389,  0.2209882 ])
  xipos: 

In [17]:
mj_data.energy

array([21.69863719,  0.        ])

In [29]:
pot_energy = 0 
for i in range(2):
    pot_energy += -mj_model.body_mass[i]*mj_data.xipos[i]@mj_model.opt.gravity
    

In [30]:
pot_energy

21.698637187527698

In [20]:
mj_data.energy

array([21.69863719,  0.        ])

In [21]:
mj_data.xpos

array([[ 0.        ,  0.        ,  0.        ],
       [-0.05734004,  0.54043218,  1.69709864]])

In [22]:
mj_model.opt.gravity

array([ 0.  ,  0.  , -9.81])

In [23]:
mujoco.mj_energyPos(mj_model,mj_data)

In [24]:
mj_model.body_iquat

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.47776755,  0.47776755, -0.52128511,  0.52128511]])

In [25]:
mj_model.body_inertia[1]

array([0.06071129, 0.0364684 , 0.0254117 ])

In [26]:
mj_data.xpos

array([[ 0.        ,  0.        ,  0.        ],
       [-0.05734004,  0.54043218,  1.69709864]])

In [27]:
mj_data.qpos

array([-0.05734004,  0.54043218,  1.69709864,  0.48406141, -0.06219627,
       -0.8678923 ,  0.09262357])

In [28]:
mujoco.mj_objectVelocity

<function mujoco._functions.PyCapsule.mj_objectVelocity>